## OpenAI's Chat Completion API

OpenAI의 Chat Completion API는 GPT-3와 GPT-4와 같은 모델을 활용하여 대화 기반 응답을 생성하는 API 
이 API를 이용해 사용자가 입력한 대화 텍스트에 대해 모델이 적절한 응답을 생성할 수 있음
주요 기능과 사용법은 다음과 같음:

### 엔드포인트:

API 호출은 /v1/chat/completions 엔드포인트를 사용함.

### 요청 형식:

요청은 주로 JSON 형식으로 이루어지며, 다음과 같은 필드를 포함합니다:

- model: 사용할 모델의 이름(e.g., gpt-3.5-turbo).
- messages: 대화의 히스토리를 나타내는 리스트. 각 메시지는 역할(role)과 내용(content) 포함. 역할은 user, assistant, system이 있음
- max_tokens: 생성할 응답의 최대 토큰 수.
- temperature: 응답의 창의성 수준을 조절하는 값 (0에서 1 사이의 실수). 낮을수록 보수적인 응답을 생성하고, 높을수록 창의적인 응답 생성함
- top_p: 응답의 다양성을 조절하는 또 다른 매개변수 (0에서 1 사이의 실수). 낮을수록 높은 확률의 단어에 집중하여 응답 생성

```
{
  "model": "gpt-3.5-turbo",
  "messages": [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "How do I make a cup of tea?"}
  ],
  "max_tokens": 150,
  "temperature": 0.7
}
```

### 응답 형식:

응답은 JSON 형식으로 반환되며, 주로 다음과 같은 필드를 포함합니다:
- id: 생성된 응답의 고유 ID.
- object: 응답 객체의 유형 (일반적으로 chat.completion).
- created: 응답이 생성된 타임스탬프.
- model: 사용된 모델의 이름.
- choices: 생성된 응답의 리스트. 각 응답은 메시지의 역할과 내용을 포함합니다.
- usage: 요청의 토큰 사용량을 나타내는 정보 (입력 토큰 수, 출력 토큰 수, 총 토큰 수).

```
{
  "id": "chatcmpl-6x5eQ4Px5mL1e7Q4e5R9j8X9A",
  "object": "chat.completion",
  "created": 1675277923,
  "model": "gpt-3.5-turbo",
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "To make a cup of tea, start by boiling some water. Then, place a tea bag in a cup and pour the hot water over it. Let it steep for a few minutes, remove the tea bag, and add any desired sweeteners or milk. Enjoy your tea!"
      },
      "finish_reason": "stop",
      "index": 0
    }
  ],
  "usage": {
    "prompt_tokens": 20,
    "completion_tokens": 44,
    "total_tokens": 64
  }
}
```
 
이 API는 챗봇, 고객 지원, 개인 비서 등 다양한 응용 프로그램에서 대화형 응답을 생성하는 데 사용될 수 있음  
API를 활용하여 다양한 시나리오에 맞춘 자연어 응답을 생성할 수 있음


## OpenAI의 공식 openai 패키지를 사용을 이용한 Chat Completion API 사용법

1. 패키지 설치
먼저 openai 패키지를 설치해야 합니다. 터미널이나 명령 프롬프트에서 다음 명령을 실행함
```
pip install openai
```

In [1]:
import openai

openai.__version__

'1.37.1'

2. API 키 설정
OpenAI API 키를 사용해야함. 이 키는 OpenAI 웹사이트에서 생성할 수 있음.

In [2]:
from openai import OpenAI

import os
from dotenv import load_dotenv

# .env 파일 경로 지정 
load_dotenv()

# Open AI API 키 설정하기
api_key = os.environ.get('OPENAI_API_KEY')


client = openai.OpenAI(api_key=api_key)

In [3]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Hello!"}
    ]
)

completion.choices[0].message.content

'Hello! How can I assist you today?'

In [4]:
from string import Template

prompt = Template("""
다음은 사용자가 작성한 제목입니다.
비속어를 제거하여 제목을 다시 작성해 주세요.
제목의 의미는 유지하도록 해주세요.
비속어가 없을 경우에는 제목을 그대로 사용합니다.

$title
""")

In [5]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt.substitute(title="벗꽃이 한참인 윤중로에 년놈들이 그득그득...")}
    ],
    temperature=1,
    max_tokens=128
)

completion.choices[0].message.content

'벗꽃이 한참인 윤중로에 사람들이 많이 모여...'

In [6]:
def refine_title(client, title, temperature=1, max_tokens=256):
    prompt = Template("""
    다음은 사용자가 작성한 제목입니다.
    비속어를 제거하여 제목을 다시 작성해 주세요.
    제목의 의미는 유지하도록 해주세요.
    비속어가 없을 경우에는 제목을 그대로 사용합니다.

    $title
    """)
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt.substitute(title=title)}
        ],
        temperature=temperature,
        max_tokens=max_tokens
    )

    return completion.choices[0].message.content

In [7]:
refine_title(client, """벗꽃이 한참인 윤중로에 년놈들이 그득그득...""")

'벚꽃이 만발한 윤중로에 사람들이 북적북적...'

In [8]:
def summarize(client, text, temperature=1, max_tokens=256):
  prompt = Template("""
  다음 기사를 요약해주세요.
  기사:
  $text
  """)
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt.substitute(text=text)}
    ],
    temperature=temperature,
    max_tokens=max_tokens
  )

  return completion.choices[0].message.content

In [9]:
def summarize(client, temperature=1, max_tokens=256, **kwargs):
  prompt = Template("""
  다음 기사를 요약해주세요.
  기사:
  $text1
  $text2
  """)
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt.substitute(**kwargs)}
    ],
    temperature=temperature,
    max_tokens=max_tokens
  )

  return completion.choices[0].message.content

In [10]:
article1 = """
한국 축구 레전드 박지성(43) 전북 테크니컬 디렉터의 '작심 발언'에도 대한축구협회(KFA) 결정에 변화는 없었다.
박지성은 12일 서울 국립현대미술관에서 열린 기획 프로그램 'MMCA 플레이: 주니어 풋살'을 끝낸 뒤 취재진과 인터뷰를 진행했다. 그는최근 홍명보 감독 선임 과정과 관련해 대한축구협회(KFA)를 향해 쓴소리를 아끼지 않았다.
KFA는 지난 7일 "축구국가대표팀 차기 감독에 홍명보 감독 울산HD 감독을 내정했다"라고 알렸다. 뒤이어 13일 KFA는 "이사회 승인을 통해 홍명보 축구국가대표팀 감독을 공식 선임했다. 홍명보 감독은 코칭스태프 구성에 들어간다"라고 공식 발표했다.
위르겐 클린스만 감독 경질 이후 5개월 동안 감독을 찾아 나선 KFA는 전력강화위원회를 꾸려 수많은 외국인 감독과 접촉했고 실제로 한국 감독직에 크게 관심을 보인 이도 있었다. 하지만 KFA는 홍명보 감독을 택했다.
논란이 많았던 결정이다. 전력강화위원으로 활동했던 전 국가대표 축구선수 박주호는 본인의 유튜브 채널을 통해 한 시간 가량 열변을 토하며 절차상 문제를 제기했다.
KFA는 이를 왜곡된 주장이라고 주장하며 법적 대응까지 검토하겠다고 밝혔다. 박주호 뿐만 아니라 이영표 등 여러 축구계 인사들이 KFA의 감독 선임 과정에서 생긴 문제를 지적했다.
박지성의 생각도 크게 다르지 않았다. 평소 자신의 생각과 의견을 잘 드러내지 않는 박지성이었지만, 이날은 달랐다.
박지성은 "첫 번째로 드는 감정은 슬픔"이라며 "한국에서 축구를 시작했고, 아직도 축구라는 분야에 있다. 하지만 '우리가 이것밖에 되지 않았나'하는 생각이 가장 먼저 들었다"라며 KFA를 향한 실망감을 이야기했다.
그는 "둘째로 이렇게 될 수밖에 없었는가에 대한 아쉬움이 너무나 크다. 축구인으로서 너무 슬픈 상황을 맞이하고 있고, 마음이 상당히 아픈 상태"라고 전했다.
박지성은 "가장 슬픈 건 뭐 하나 확실한 답이 없다는 것"이라고 짚었다. 그는 "2002 월드컵을 통해 한국 축구는 상당히 변했고, 앞으로 상당히 많이 변해갈 것이라는 기대가 있었다. 그런데 그때와 달라진 것이 무엇인가에 대한 답을 이렇게 받았다"라고 말했다.
뒤이어 그는 "뭐라 말할 수 없을 정도로 참담한 기분"이라며 큰 실망감을 전했다.
박지성은 "과연 어디까지 이래야 하는 것인가. 협회에서 일한다는 게 누구에게나 의미가 있고, 누구나 하고 싶어 하는 일이 되어야 한다. 지금은 아무도 하고 싶지 않은 일이 돼가고 있다. '저 안에 들어가면 제대로 된 일을 할 수 있을까'하는 의문을 남겼다"라며 KFA의 신뢰가 무너졌다고 지적했다.
특유의 담담한 말투였지만, 내용은 거침없었다. 박지성은 "현재 나온 이야기들로만 봤을 땐 도무지 나올 수 없는 답을 안고 있다. 왜 그럴 수밖에 없었는지 설명이 필요하다. 분명히 나름의 이유가 있을 거란 생각은 있지만, 내부 사람이 아니면 알 수 없는 문제"라고 꼬집었다.
"""

In [11]:
article2 = """그는 "지금 이 상황을 아무런 해결책 없이 넘어가면 안 된다"라고 주장하면서 "언제 어떻게 이 해결책을 제시하고 다음 단계로 나아갈지가 필요하다. 여기서 멈춰서 한국 축구가 끝나는 걸 모두가 바라볼 수는 없을 것"이라고 호소했다.
박지성은 절차 문제를 언급했다. 그는 "결과적으로는 진실이 답이다. 진실을 알아야 해결책을 가질 수 있다. 이미 협회의 신뢰는 떨어졌고, 회복하기까지는 오랜 시간이 걸릴 것이다. 그 회복의 시작을 위해서는 진실을 말하고, 사실대로 말하면서 앞으로 나아가야 되지 않을까. 절차대로 밟아서 감독을 선임하겠다는 약속 자체가 무너졌기 때문에 당장 사실을 말하더라도 받아들일 사람이 많지 않을 것"이라고 꼬집었다.
최근 감독 선임 과정에 대해 폭로한 박주호를 언급하기도 했다. 그는 "상당히 많은 무력감을 느꼈을 것이라는 생각이 들었다. (박주호) 본인의 의견이 100% 회의에서 받아들여질 수 없는건 사실이다. 하지만 결국 안에서 얘기했던 절차대로 진행되지 않았고, 그 자리에 있을 필요가 없다고 느껴지는 무력감은 상당히 컸을 것"이라며 후배를 걱정했다.
박지성은 "행정절차가 투명하지 않고 올바른 시스템이 없으면 좋은 인재들을 제대로 활용하지 못한다"라며 "그 인재들을 제물로 써야 하는 상황이 벌어지고 있는 부분이 가장 안타까운 결과"라고 말했다.
한국 축구는 1992년생 손흥민과 이재성을 필두로 '황금세대'를 맞았다. 1996년생 김민재와 황희찬, 황인범까지 포진돼 있다. 김민재, 황희찬, 황인범은 만나이로 27세~28세로 전성기에 접어든 나이이며 손흥민은 만 32세로 전성기를 누리는 나이다. 박지성은 이들의 전성기를 낭비하는 것에 대해 아쉬움을 표했다.
박지성은 정몽규 KFA 회장에 대한 비판도 아끼지 않았다. 그는 "체계가 완전히 무너졌다. 바로 세우고 나아갈 것이란 기대는 5개월 전이 마지막 아니었나"라고 말했다.
그는 정몽규 회장의 사퇴 여론에 대해 "상당히 어려운 게 사실"이라고 말했다. 이어 "규정이 없는 상황에서 외부의 압력으로 어디까지 할 수 있을지는 솔직히 모르겠다. 회장이 스스로 선택을 하셔야 하는 상황인 것만은 분명한 사실"이라고 설명했다.
박지성은 후배들에 대한 미안함과 대표팀에 대한 우려를 남겼다. 그는 "선임 번복은 협회와 홍명보 감독님의 결정이 남아있다. 하지만 지금 상황에서 쉽사리 앞으로 나아갈 수 있을지는 걱정"이라며 "(후배들에게) 가장 큰 생각은 미안하다는 것이다. 선배로서 조금이나마 좋은 환경에서 실력을 뽐낼 수 있게 만들어줬어야 하는데. 한국 축구 역사에서 가장 좋은 선수들로 구성된 지금 그걸 뒷받침할 수 없는 상황"이라고 말했다.
이어 "우리 축구인들뿐만 아니라 팬들 역시도 그 부분이 가장 아쉽다고 생각할 것"이라고 말했다.
박지성의 목소리에도 KFA의 결정에 변함은 없었다. KFA는 "지난 10일부터 12일까지 사흘 동안 홍명보 감독 선임 건으로 2024년 4차 이사회 서면결의를 실시했다. 해당 건은 23명 중 21명의 찬성으로 승인됐다"라고 알렸다.
공식적으로 대표팀 사령탑 업무에 착수하게 된 홍명보 감독은 대표팀 코칭스태프 구성에 들어간다. 특히 세계축구의 흐름 파악과 분석에 도움이 될 외국인 코치의 경우 후보자를 체크하고, 유럽에서 면담을 진행할 계획이다.
"""

In [12]:
summarize(client, max_tokens=500, text1=article1, text2=article2)

'한국 축구 레전드인 박지성은 KFA의 감독 선임 과정과 관련해 실망과 아쉬움을 토로했지만, KFA는 홍명보 감독의 선임을 공식화했다. 논란이 있었던 결정에 대한 박지성의 비판은 KFA에 변화를 야기하지는 못했다. 홍명보 감독은 대표팀 코칭스태프를 구성하고, 외국인 코치 후보자를 체크하고 유럽에서 면담할 계획이다. 한편 박지성은 KFA의 결정에 대한 진실과 투명성을 강조하며 해결책을 모색할 필요가 있다고 주장했다.'

In [13]:
def complete_chat(client, user_prompt, model="gpt-3.5-turbo", temperature=1, max_tokens=256, **kwargs):

  completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": user_prompt.substitute(**kwargs)}
    ],
    temperature=temperature,
    max_tokens=max_tokens
  )

  return completion.choices[0].message.content

In [14]:
prompt = Template("""
  다음 기사를 요약해주세요.
  기사:
  $text1
  $text2
  """)

In [15]:
complete_chat(client, prompt, text1=article1, text2=article2)

'박지성 전북 테크니컬 디렉터는 KFA의 홍명보 감독 선임 결정에 실망을 표현했고, KFA는 이에 대해 변화 없음을 밝혔다. 박지성은 또한 KFA의 절차 문제와 현재 상황에 대한 우려를 피력했으며, 후배들과 팬들에게 미안함을 표하며 지적했다. 그러나 KFA는 결정을 고수하고 홍명보 감독을 공식 선임했다.'

In [20]:
prompt = Template("""
다음 기사를 요약해주세요.
기사:
$text
""")

In [21]:
complete_chat(client, prompt, max_tokens=512, text=article1+article2)

"한국 축구 레전드 박지성이 한국 축구협회(KFA)의 감독 선임 과정과 관련한 논란을 지적하며 실망을 토로했지만, KFA는 결정에 변화가 없다고 밝혔다. 박주호 등 축구계 인사들도 감독 선임 절차에 대해 문제를 지적했고, 박지성은 KFA의 결정에 실망을 표명하며 협회의 신뢰가 무너졌다고 지적했다. 또한 그는 정몽규 KFA 회장에 대한 비판과 후배들에 대한 미안함을 표현했다. 한국 축구는 현재 '황금세대'의 전성기를 낭비하는 상황이며, 박지성은 협회의 투명성과 시스템 개선을 요구했다. 최근 홍명보 감독이 선임되었지만, 박지성의 비판에도 KFA는 변화를 이루지 않았다."

In [22]:
prompt = Template("""
다음 기사에 대한 제목을 만들어 주세요.
기사:
$text
""")

complete_chat(client, prompt, max_tokens=512, text=article1+article2)

"박지성의 '작심 발언'에도 KFA 결정 변화는 없어"

In [27]:
def complete_chat(client, user_prompt, system_prompt=None, assistant_prompt=None, model="gpt-3.5-turbo", temperature=1, max_tokens=256, **kwargs):

  messages = []
  if user_prompt:
    messages.append({"role": "user", "content": user_prompt.substitute(**kwargs)})
  if system_prompt:
    messages.append({"role": "system", "content": system_prompt})
  if assistant_prompt:
    messages.append({"role": "assistant", "content": assistant_prompt})

  completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature,
    max_tokens=max_tokens
  )

  return completion.choices[0].message.content

In [28]:
prompt = Template("""
다음 기사에 대한 제목을 만들어 주세요.
기사:
$text
""")

complete_chat(client, prompt, max_tokens=512, text=article1+article2)

"박지성의 '작심 발언'에도 KFA 결정 변화는 없어"

In [29]:
system_prompt = """
규칙:
- 비속어의 수준이 상인 경우 비속어를 순화하여 제목을 출력하세요.
- 비속어의 수준이 중이거나 하인 경우 원래 제목을 출력하세요.
"""

user_prompt = Template("""
사용자가 작성한 제목을 다듬는 작업을 진행합니다.

비속어의 수준:
상: 극도로 모욕적이고, 공격적인 표현 예) 시발놈, 개새끼, 씨발, 광탈
중: 다소 공격적이고 무례한 표현으로, 상대방에게 불쾌감을 줄 수 있는 표현 예) 병신, 미친놈
하: 일상적인 대화에서 가볍게 사용되며, 흔히 친한 친구들 사이에서는 장난스럽게 쓰이기도 하는 표현 예) 멍청이, 똥개, 바보

절차:
1. 제목의 어떤 내용이 비속어인지 나열하세요.
2. 비속어라고 선정한 이유를 말하고, 그에 따라 수준을 상,중,하로 평가하세요.

출력포맷:
1. 비속어: {찾은 비속어}
2. 선정한 이유: {비속어라고 선정한 이유}
3. 비속어 수준: {비속어 수준}

{제목}

제목:
$title
""")

In [30]:
message = complete_chat(client, user_prompt=user_prompt, system_prompt=system_prompt, title="한국 축구 AFC컵 광탈")
print(message)

1. 비속어: 광탈
2. 선정한 이유: "광탈"은 극도로 모욕적이고 부정적인 뜻을 내포하고 있어 상위 비속어로 분류됩니다.
3. 비속어 수준: 상

제목:
한국 축구 AFC컵 광탈
